In [1]:
'''Complete Information DQN (CIdqn)
'''

import glob
import imp
import math
import gc
import os
from sre_constants import SUCCESS
import time
import datetime
import pybullet as p
import cv2
import numpy as np
from graphviz import Digraph
import argparse
import random
import torch
import matplotlib.pyplot as plt
from time import sleep
import copy

import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


from Config.constants import (
    GRIPPER_PUSH_RADIUS,
    PIXEL_SIZE,
    PUSH_DISTANCE,
    WORKSPACE_LIMITS,
    TARGET_LOWER,
    TARGET_UPPER,
    orange_lower,
    orange_upper,
    BG_THRESHOLD,
    MIN_GRASP_THRESHOLDS
)

from Environments.environment_sim import Environment
import Environments.utils as env_utils
from V1_destination_prediction.Test_cases.tc1 import TestCase1

from create_env import get_push_start, get_max_extent_of_target_from_bottom


from collections import namedtuple, deque

from V2_next_best_action.models.dqn_v2 import pushDQN2

torch.cuda.empty_cache()

pybullet build time: Oct 14 2022 01:09:34


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
n_observations = 6 # 3 for initial state, 3 for goal state
n_actions = 16 # 16 push + 1 grasp

policy_net = pushDQN2(n_observations, n_actions, use_cuda=True).to(device)

checkpoint = torch.load('/home/vishal/Volume_E/Active/Undergrad_research/ICRA22/codebases/TEMP/Mid-Level-Planner/V2_next_best_action/models/model_checkpoints/model9/5350.pt')
policy_net.load_state_dict(checkpoint)
policy_net.eval()

pushDQN2(
  (layer1): Linear(in_features=6, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=16, bias=True)
)

In [4]:
def select_action(state):
    '''Select the next best action 
    state: tensor(shape=(6))
    '''
    with torch.no_grad():
        return policy_net(state).max(1)[1].view(1, 1)

def get_reward(prev_state, current_state):
    '''
    prev_state: (x1, y1, theta1, x2, y2, theta2)
    current_state: (x3, y3, theta3, _, _, _)
    '''
    reward = np.exp(-1 * np.sqrt(np.linalg.norm(current_state[0:3] - prev_state[3:6])))
    return reward

def get_reward2(prev_state, current_state):
    '''
    prev_state: (x1, y1, theta1, x2, y2, theta2)
    current_state: (x3, y3, theta3, _, _, _)
    '''

    print("Rewarding ----------------------------")
    pos_diff = np.linalg.norm(prev_state[0:2] - prev_state[3:5]) - np.linalg.norm(current_state[0:2] - prev_state[3:5])
    orn_diff = np.linalg.norm(prev_state[2:3] - prev_state[5:6]) - np.linalg.norm(current_state[2:3] - prev_state[5:6])
    reward = pos_diff + 0.1*orn_diff  #np.linalg.norm(prev_state[0:3] - prev_state[3:6]) - np.linalg.norm(current_state[0:3] - prev_state[3:6]) # prev distance - current distance
    print(f"Position Diff: {pos_diff}\tOrn Diff: {0.1*orn_diff}\nReward Aggregate: {reward}")
    return reward

def get_reward4(prev_state, current_state):
    '''
    prev_state: (x1, y1, theta1, x2, y2, theta2)
    current_state: (x3, y3, theta3, _, _, _)
    '''

    # print("Rewarding ----------------------------")
    pos_diff = np.linalg.norm(prev_state[0:2] - prev_state[3:5]) - np.linalg.norm(current_state[0:2] - prev_state[3:5])
    orn_diff = np.linalg.norm(prev_state[2:3] - prev_state[5:6]) - np.linalg.norm(current_state[2:3] - prev_state[5:6])
    reward = pos_diff + 0.1*orn_diff  #np.linalg.norm(prev_state[0:3] - prev_state[3:6]) - np.linalg.norm(current_state[0:3] - prev_state[3:6]) # prev distance - current distance
    # print(f"Position Diff: {pos_diff}\tOrn Diff: {0.1*orn_diff}\nReward Aggregate: {reward}")
    return reward

In [5]:
from Environments.utils import sample_goal, get_pose_distance
# def sample_goal(target_pos, target_orn):
#     target_euler = p.getEulerFromQuaternion(target_orn)
#     target_yaw = target_euler[2]

#     goal_pos = target_pos + np.random.uniform(low=-0.15, high=0.15, size=(3,))
#     goal_pos[2] = target_pos[2]
#     goal_yaw = target_yaw + np.random.uniform(low=0, high=2*np.pi)
#     goal_euler = np.array(target_euler)
#     goal_euler[2] = goal_yaw

#     goal_orn = p.getQuaternionFromEuler(goal_euler)

#     return goal_pos, goal_orn

# def get_pose_distance(target_pos, target_orn, marker_pos, marker_orn):
#     target_euler = p.getEulerFromQuaternion(target_orn)
#     target_yaw = target_euler[2]

#     marker_euler = p.getEulerFromQuaternion(marker_orn)
#     marker_yaw = marker_euler[2]

#     t_pose = np.array([target_pos[0], target_pos[1], target_yaw], dtype=float)
#     m_pose = np.array([marker_pos[0], marker_pos[1], marker_yaw], dtype=float)

#     return np.linalg.norm(t_pose - m_pose)

In [6]:
push_directions = [0, np.pi/8, np.pi/4, 3*np.pi/8, 
                    np.pi/2, 5*np.pi/8, 3*np.pi/4, 7*np.pi/8, 
                    np.pi, 9*np.pi/8, 5*np.pi/4, 11*np.pi/8,  
                    3*np.pi/2, 13*np.pi/8, 7*np.pi/4, 15*np.pi/8] # 16 standard directions

max_timesteps = 250
max_per_episode_timesteps = 50
timestep = 0


env = Environment(gui=True)
threshold_d = 0.01 # Threshold pose distance

while timestep < max_timesteps:
    env.reset()

    testcase1 = TestCase1(env)
    body_ids, success = testcase1.sample_test_case(bottom_obj='random')

    target_pos, target_orn = p.getBasePositionAndOrientation(body_ids[1])
    marker_pos, marker_orn = None, None
    goal_suc = False
    while not goal_suc:
        marker_pos, marker_orn = sample_goal(target_pos, target_orn)
        marker_obj, goal_suc = testcase1.add_marker_obj(marker_pos, marker_orn, half_extents=testcase1.current_target_size/2)
    body_ids.append(marker_obj)

    target_euler = p.getEulerFromQuaternion(target_orn)
    marker_euler = p.getEulerFromQuaternion(marker_orn)
    cur_target_st = np.array([target_pos[0], target_pos[1], target_euler[2]], dtype=np.float64)
    cur_target_goal = np.array([marker_pos[0], marker_pos[1], marker_euler[2]], dtype=np.float64)
    cur_state = np.hstack((cur_target_st, cur_target_goal))
    state = {
        'cur_state': torch.tensor(cur_state, dtype=torch.float, device=device).unsqueeze(0),
    }

    print(f"Timestep: {timestep}\tTarget: {cur_target_st}\tGoal: {cur_target_goal}------------------")
    pose_d = get_pose_distance(target_pos, target_orn, marker_pos, marker_orn)
    print(f"Pose distance: {pose_d}")

    per_episode_timesteps = 0
    while (pose_d > threshold_d) and per_episode_timesteps < max_per_episode_timesteps:
        action = select_action(state['cur_state'])
        print(f"Action: {push_directions[action.item()]}")
        color_image, depth_image, _ = env_utils.get_true_heightmap(env)

        new_target_pos, new_target_orn = None, None
        new_state=None
        if action.item() in range(0, 16):
            temp = cv2.cvtColor(color_image, cv2.COLOR_RGB2HSV)
            target_mask = cv2.inRange(temp, TARGET_LOWER, TARGET_UPPER)
            push_dir = push_directions[action.item()] # Sample push directions
            push_start, push_end = get_push_start(push_dir, target_mask, body_ids[1])
            env.push(push_start, push_end)

            # collect reward
            new_target_pos, new_target_orn = p.getBasePositionAndOrientation(body_ids[1])
            target_euler = p.getEulerFromQuaternion(new_target_orn)

            new_target_st = np.array([new_target_pos[0], new_target_pos[1], target_euler[2]], dtype=np.float)
            # new_target_goal = new_target_st + np.random.uniform(low=[-5, -5, -2*np.pi], high=[5, 5, 2*np.pi], size=(3,))
            new_state = np.hstack((new_target_st, cur_target_goal))
            reward = get_reward2(current_state=new_state, prev_state=state['cur_state'].squeeze().cpu().numpy())
            print(f"Reward: {reward}")
        else:
            print("Invalid action type!!!!!!!")
            exit()
        
        pose_d = get_pose_distance(new_target_pos, new_target_orn, marker_pos, marker_orn)

        next_state = {
            'cur_state': torch.tensor(new_state, dtype=torch.float, device=device).unsqueeze(0)
        }
        state=next_state
        timestep+=1
        per_episode_timesteps += 1

        if timestep >= max_timesteps:
            print("The end!")
            break

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1050 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 510.85.02
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 510.85.02
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1050 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
Timestep: 0	Target: [ 0.34799836  0.12984701 -0.34883955]	Goal: [0.38709285 0.13171186 1.56367126]------------------
Pose distance: 1.912911250954479
Action: 1.57079632679

: 

: 